In [1]:
import pandas as pd 
import  pandas as pd 


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder ,MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipelines
from sklearn.feature_selection import SelectKBest ,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv('train.csv')


In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [99]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace =True)

In [100]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [101]:
X_train ,X_test ,y_train,y_test = train_test_split(df.drop(columns =['Survived']),
                                                   df['Survived'],
                                                   test_size = 0.2,
                                                   random_state = 42)

In [102]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [103]:
y_train.sample(5)

397    0
523    1
704    0
476    0
866    1
Name: Survived, dtype: int64

In [104]:
# imputation transformer 
trf1 = ColumnTransformer([
    
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embraked',SimpleImputer(strategy = 'most_frequent'),[6])
    
],remainder= 'passthrough')

In [105]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
], remainder='passthrough')


In [51]:
trf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 [1, 6])])

In [106]:
# scaling 
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [107]:
# fearure selection 
trf4 = SelectKBest(score_func=chi2,k=8)

In [108]:
# train the model 

trf5 = DecisionTreeClassifier()

# create pipline

In [145]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

# pipeline Vs make_pipeline 

In [144]:
# alternate syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [146]:
# train 
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embraked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f4199e99510>)),
                ('trf5', DecisionTreeClassifier())])

In [147]:
# code herer 
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embraked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7f4199e99510>),
 'trf5': DecisionTreeClassifier()}

In [136]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [137]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_


array(['S'], dtype=object)

# Explore pipeline 

In [148]:
# Display spipeline 
from sklearn import set_config
set_config(display ='diagram')

In [149]:
# predict 
y_pred = pipe.predict(X_test)

In [150]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

# cross validation using Pipeline 

In [154]:
# cross validation using cross_cal_score , it is used it many time train_test_split and get the accuracy this work can we done using pipeline
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv =5,scoring = 'accuracy').mean()

0.6391214419383433

#  GridSearch using Pipeline

In [163]:
# gridsearchcv ,it will chose the best one from all the conditions of the algorithm 
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [164]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embraked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7f4199e99510>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [165]:
grid.best_score_

0.6391214419383433

In [167]:
grid.best_params_

{'trf5__max_depth': 2}

In [169]:
import pickle
pickle.dump(pipe,open('models/titanic_piplen.pkl','wb'))